In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

In [2]:
np

<module 'numpy' from '/opt/conda/lib/python3.7/site-packages/numpy/__init__.py'>

In [60]:
# Reading input

articles = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv')
customers = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/customers.csv')
transactions_train = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')

In [ ]:
transactions_train['max_purc_date'] = transactions_train.groupby('customer_id')['t_dat'].transform('max')

transactions_train['min_purc_date'] = transactions_train.groupby('customer_id')['t_dat'].transform('min')

transactions_train['total_visits'] = transactions_train.groupby('customer_id')['t_dat'].transform('nunique')

transactions_train['total_item_bought'] = transactions_train.groupby('customer_id')['article_id'].transform('nunique')

transactions_train['max_year'] = transactions_train['max_purc_date'].str.slice(0,4).astype(int)

In [ ]:
transactions_train_filt = transactions_train[transactions_train['min_year'] == 2020]

transactions_train_filt = transactions_train_filt[transactions_train_filt['total_visits'] >= 10]

transactions_train_filt = transactions_train_filt[transactions_train_filt['total_item_bought'] >= 10]

In [ ]:
transactions_train_filt['product_reach'] = transactions_train_filt.groupby('article_id')['customer_id'].transform('nunique')

transactions_train_filt = transactions_train_filt[transactions_train_filt['product_reach'] > 8]

In [32]:
## Product purchased atleast twice

transactions_train_filt['repeat_purchase'] = transactions_train_filt.groupby(['customer_id','article_id'])['t_dat'].transform('nunique')

In [33]:
transactions_train_filt = transactions_train_filt[transactions_train_filt['repeat_purchase'] > 1]

In [26]:
transactions_train_filt['t_dat_datetime']  = pd.to_datetime(transactions_train_filt['t_dat'])

In [34]:
product_purchase_cycle = transactions_train_filt[['customer_id','article_id','t_dat_datetime']].\
sort_values(['customer_id','article_id','t_dat_datetime']).drop_duplicates()

In [39]:
product_purchase_cycle['t_dat_datetime_lag'] = product_purchase_cycle.\
groupby(['customer_id','article_id'])['t_dat_datetime'].shift(1)

In [42]:
product_purchase_cycle['purchase_cycle'] = product_purchase_cycle['t_dat_datetime'] - product_purchase_cycle['t_dat_datetime_lag']

In [54]:
Average_Purchase_Cycle = product_purchase_cycle[~(product_purchase_cycle['purchase_cycle'].isnull())].\
groupby(['customer_id','article_id']).agg({"purchase_cycle":"median"}).reset_index()

In [57]:
Average_Purchase_Cycle.agg({"purchase_cycle":"min","purchase_cycle":"max"})

purchase_cycle   733 days
dtype: timedelta64[ns]

In [7]:
sample_submission = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')

In [8]:
submission_train_customer = pd.merge(sample_submission['customer_id'], 
                                     transactions_train_filt,
                                     left_on = 'customer_id',
                                     right_on = 'customer_id',
                                     how = 'inner'
                                    )

In [9]:
import gc
del sample_submission
gc.collect()

23

In [10]:
del transactions_train
gc.collect()

23

In [11]:
submission_train_customer =  pd.merge(   submission_train_customer,
          customers[['customer_id','age']],
                                     left_on = 'customer_id',
                                     right_on = 'customer_id',
                                     how = 'inner'
                                    )

In [12]:
age_wise_top_products = submission_train_customer.groupby(['age','article_id']).agg({'customer_id':'nunique'}).reset_index()

age_wise_top_products['rank'] = age_wise_top_products.groupby('age')['customer_id'].rank(method="first",ascending=True)

selected_products = age_wise_top_products[age_wise_top_products['rank']<= 12][['age','article_id']]

In [13]:
selected_products['article_id'] = ' 0'+selected_products['article_id'].astype('str')

In [14]:
selected_products['age'].unique()

array([16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26., 27., 28.,
       29., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39., 40., 41.,
       42., 43., 44., 45., 46., 47., 48., 49., 50., 51., 52., 53., 54.,
       55., 56., 57., 58., 59., 60., 61., 62., 63., 64., 65., 66., 67.,
       68., 69., 70., 71., 72., 73., 74., 75., 76., 77., 78., 79., 80.,
       81., 82., 83., 84., 85., 86., 87., 88., 89., 90., 91., 92., 95.,
       96., 97., 98.])

In [15]:
age_wise_top_12_products = pd.DataFrame(selected_products.groupby(['age']).article_id.sum().reset_index())

In [16]:
sample_submission = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')

In [17]:
sample_submission.shape

(1371980, 2)

In [18]:
sample_submission = pd.merge(sample_submission['customer_id'], customers[['customer_id','age']]
         ,left_on = 'customer_id'
         ,right_on = 'customer_id'
         ,how = 'left'
        )

In [19]:
sample_submission.shape

(1371980, 2)

In [20]:
sample_submission = pd.merge(sample_submission,
         age_wise_top_12_products,
         left_on = 'age',
         right_on = 'age',
         how = 'left'
        )

In [21]:
sample_submission['article_id'].fillna( "0706016001 0706016002 0372860001 0399223001 0464297007 0160442007 0610776002 0156231001 0399256001 0372860002 0706016003 0759871002", inplace=True)

In [22]:
sample_submission.drop('age', axis=1, inplace=True)

In [23]:
sample_submission = sample_submission.rename(columns = {'article_id':'prediction'})

In [24]:
sample_submission.to_csv('submission.csv',index=False)